# Returning values and referential transparency

We now continue our journey to capture independent units of logic in the form of functions. So far, we have defined basic functions, shown how the stack allows us to define a local environment, called the scope, in which functions may save their own private data without conflicting with other functions, and finally we have shown that, with such a mechanism, functions may nest calls to each other without much danger.

Unfortunately, we still have to tackle one last gray area: functions that *write* to the same global variable can produce a clash, which is an unwanted dangerous behaviour. For example, consider the functions `incr := <k => x := x + k>` and `mul := <k => x := x * k>`. Both functions write to the global variable `x` in order to be able to "leave a trace" of what they have computed. This makes sense given our framework so far, since a function that leaves no trace anywhere in the state is useless, so some form of result must remain behind after the function is done. The issue is that for the two functions we have just seen, the fact that they both act on `x` induces a coordination between them that might not be desirable. For example, if we execute the following program:

```
x := 1
incr(3)
mul(2)
```

then we obtain a final state where `x := 8`, while if we execute the very similar program with the two function calls switched:

```
x := 1
mul(2)
incr(3)
```

then we obtain a final state where `x` has a different value (`x := 5`).

Even worse, consider the more complex example of drawing a hollow square. Following our recent philosophy of isolating concepts, we begin by defining a few key ideas such as how to "draw a full line" and "draw a hollow line":

```
draw_full_line := () => s := "*****"
draw_hollow_line := () => s := "*   *"
...
```

Notice how both functions simply put inside `s` their result. We might now also want to define the function for drawing the whole hollow square, without thinking about drawing the individual lines anymore, since those are "solved problems" and we do not need to tackle them anymore:

```
...

draw_square := () =>
  s := ""
  draw_full_line()
  s := s + ?
  draw_hollow_line()
  s := s + ?
  draw_hollow_line()
  s := s + ?
  draw_hollow_line()
  s := s + ?
  draw_full_line()
  s := s + ?
```

Unfortunately we encounter a problem: if we want to keep following the same convention of "putting the result into `s`", then we get a serious clash. Since `s` is also used by the line-drawing functions, everytime we draw a line then `s` gets effectively reset. We might "solve" this issue by backing `s` up before drawing each line or by using a temporary variable to accumulate the results before moving them to `s`, for example by restructuring the function above as follows:

```
...

draw_square := () =>
  tmp_s := ""
  draw_full_line()
  tmp_s := tmp_s + s + "\n"
  draw_hollow_line()
  tmp_s := tmp_s + s + "\n"
  draw_hollow_line()
  tmp_s := tmp_s + s + "\n"
  draw_hollow_line()
  tmp_s := tmp_s + s + "\n"
  draw_full_line()
  s := tmp_s + s + "\n"
```

The code we have just written would likely work, but is dangerously confusing. There is no safe way to make use of the functions `draw_full_line` and `draw_hollow_line` without explicitly keeping track of what variables they will write. This defeats the purpose of isolated units of computation, and clearly breaks isolation (also known as **encapsulation** in this kind of context). Ideally, we would like to ensure that functions are completely independent from each other, and our definition so far is weak because functions depend too much on the variables they write. A good programming language must adjust to the logical concepts we try to express, not the other way around, so we must address this issue!

## Returning
The area in which we are having trouble is the communication of the results of a function to its caller. To achieve this, our functions simply write to a previously agreed-upon location in the state, and callers will find the result there. In the previous examples about drawing, this "agreed-upon location" was variable `s`.

Because the only variables that are available to each function, caller and callee, are the global variables, sharing them across functions will produce conflicts. Such conflicts, which are broadly known as _data races_ or _race conditions_, are some of the most subtle bugs, and are often very difficult to reproduce and debug. Such a source of danger should be managed at the language level so that we do not have to regularly rely on a feature that is so easy to use incorrectly to almost be seen as a design issue of the language itself.

What we want to achieve is a safe place for each function to store its output, so that the act of _returning a result_ from a function happens in isolation from all other functions. This increases overall isolation of the different parts of our programs, and as such is a positive and desirable property.

To return a result we introduce a new statement, unsurprisingly called `return`. Its syntax is simply `return E`, where `E` is the expression that we want to return. Such an expression can be as simple (`3+x`) or as complex (`f(x) + g(y)`) as we want.

The semantics of `return` simply evaluates the expression `E` until we reach a constant value, which will then be yielded as the actual result:

- $\text{eval(<return E>, S)} \rightarrow \text{<return E'>, S'}$ where $\text{eval(<E>, S)} \rightarrow \text{<E'>, S'}$

A function might now result in something else other than just the `done` statement when it terminates. This means that the semantics of `call` must be adjusted in order to consider the scenario of a function which terminates by returning a value, which is handled by removing the `call` statement and simply yielding the value as a result:

- $\text{eval(<call(return C)>, S)} \rightarrow \text{<C>, S[stack := S[stack][t]]}$ when $\text{C}$ is a (constant) value
- $\text{eval(<call(return C; ...)>, S)} \rightarrow \text{<C>, S[stack := S[stack][t]]}$ when $\text{C}$ is a (constant) value
- $\text{eval(<call(P)>, S)} \rightarrow \text{<call(P')>, S'}$ where $\text{eval(<P>, S)} \rightarrow \text{<P'>, S'}$

Notice that thanks to the second rule for `call`, we are ignoring the rest of the function when we return, meaning that functions "short-circuit" and terminate right away when they have found the final result.

<div class="alert alert-block alert-info">
Now that we have acquired the ability for functions to communicate with the calling context by means of return, we can safely assume we will not need anymore to *write* to global variables from inside a function. This ability we are giving up is dangerous, as it can lead to very complex and unpredictable code. We needed it before defining `return`, since otherwise our functions were unable to produce any meaningful effects on the global state, but now this effect will be mediated by assigning (or further returning) the result of a function.

The new rules for writing to variables therefore reach their final version:

- $eval(<V := C>, S) \rightarrow S[stack := S[stack][h := S[stack][h][V := C]]]$ when $S[stack] \neq \{\}$
- $eval(<V := C>, S) \rightarrow S[V := C]$ otherwise

Reading from variables will still prioritize local variables, so no change in the semantics there.
</div>

## Examples of functions using `return`
Let us now consider some simple examples of drawing functions that make use of the `return` statement. The biggest change is that functions will `return` what they are supposed to draw instead of writing it to some arbitrary global variable. 

We will be working "bottom up", that is we will start by defining basic elements, and then combine them into more and more complex shapes until we reach the desired figure.

The first example will be drawing a 4x4 hollow square. We start by setting up some utility functions that draw single characters such as asterisks, newlines, and spaces:

```
asterisk := () => return "*"
space    := () => return " "
new_line := () => return "\n"
```

We now take our first step towards higher complexity by combining these basic functions together into more complex shapes. The simplest of these more complex shapes will be the whole lines, specifically the full and the hollow lines that make up the rows of the hollow square:
```
full_line := () => return asterisk() + asterisk() + asterisk() + asterisk() + new_line()
hollow_line := () => return asterisk() + space() + space() + asterisk() + new_line()
```

The next step upwards in terms of abstraction is simple and elegant, and directly provides the final result:

```
hollow_square := () => return full_line() + hollow_line() + hollow_line() + full_line()
```

|Program|State|
|:-------|:-----|
|`<asterisk := () => return "*";...>`|$\{ stack:=\{\} \}$|
|`<space := () => return " ";...>`|$\{asterisk := () \Rightarrow ..., stack:=\{\} \}$|
|`<new_line := () => return "\n";...>`|$\{asterisk := () \Rightarrow ..., space    := () \Rightarrow ...,stack:=\{\} \}$|
|`<full_line := () => return asterisk() + asterisk() + asterisk() + asterisk() + new_line();...>`|$\{asterisk := () \Rightarrow ..., space    := () \Rightarrow ...,new\_line := () \Rightarrow ...,stack:=\{\} \}$|
|`<hollow_line := () => return asterisk() + space() + space() + asterisk() + new_line();...>`|$\{asterisk := () \Rightarrow ..., space    := () \Rightarrow ...,new\_line := () \Rightarrow ...,full\_line := () \Rightarrow ...,stack:=\{\} \}$|
|`<hollow_square := () => return full_line() + hollow_line() + hollow_line() + full_line();...>`|$\{asterisk := () \Rightarrow ..., space    := () \Rightarrow ...,new\_line := () \Rightarrow ...,full\_line := () \Rightarrow ...,hollow\_line := () \Rightarrow ...,stack:=\{\} \}$|
|`<s=hollow_square();...>`|$\{asterisk := () \Rightarrow ..., space    := () \Rightarrow ...,new\_line := () \Rightarrow ...,full\_line := () \Rightarrow ...,hollow\_line := () \Rightarrow ...,hollow\_square := () \Rightarrow ...,stack:=\{\} \}$|
|`<s=call(return full_line() + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{\}\} \}$|
|`<s=call(return call(return asterisk() + asterisk() + asterisk() + asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
|`<s=call(return call(return call(return "*") + asterisk() + asterisk() + asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\}\} \}$|
|`<s=call(return call(return "*" + asterisk() + asterisk() + asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
|`<s=call(return call(return "*" + call(return "*") + asterisk() + asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\}\} \}$|
|`<s=call(return call(return "*" + "*"+ asterisk() + asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
|`<s=call(return call(return "**" + asterisk() + asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$…$|
|`<s=call(return call(return "**" + call(return "*") + asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\}\} \}$|
|`<s=call(return call(return "**" +"*"+ asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
|`<s=call(return call(return "***"+ asterisk() + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$…$|
|`<s=call(return call(return "***"+ call(return "*") + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\}\} \}$|
|`<s=call(return call(return "***"+"*" + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
|`<s=call(return call(return "****" + new_line()) + hollow_line() + hollow_line() + full_line());...>`|$…$|
|`<s=call(return call(return "****" + return "\n") + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\}\} \}$|
|`<s=call(return call(return "****" + "\n") + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
|`<s=call(return call(return "****\n") + hollow_line() + hollow_line() + full_line());...>`|$…$|
|`<s=call(return "****\n" + hollow_line() + hollow_line() + full_line());...>`|$\{…, stack:=\{h:=\{\},t:=\{\}\} \}$|
|`<…>`|$…$|
|`<s=call(return "****\n" + "*   *\n" + hollow_line() + full_line());...>`|$…$|
|`<s=call(return "****\n*   *\n" + hollow_line() + full_line());...>`|$…$|
|`<…>`|$…$|
|`<s=call(return "****\n*   *\n" + "*   *\n" + full_line());...>`|$…$|
|`<s=call(return "****\n*   *\n*   *\n" + full_line());...>`|$…$|
|`<…>`|$…$|
|`<s=call(return "****\n*   *\n*   *\n" + "****\n");...>`|$…$|
|`<s=call(return "****\n*   *\n*   *\n****\n");...>`|$…$|
|`<s="****\n*   *\n*   *\n****\n";...>`|$\{…, stack:=\{\}\}$|
|`<…>`|$\{…,s=\text{\"****\n*   *\n*   *\n****\n\", stack:=\{\}\}$|

Notice that in this example, for the very first time, we didn't need variables anywhere. This is a desirable side-effect of programming with isolated units of code, that is programs are simply built up as a composition of independent parts with negligible state usage.
    
Let us now generalize our functions so that they also accept the size of the hollow square as input. The basic functions for drawing single characters stay exactly the same, whereas drawing full and hollow lines now accepts the size of the square as a parameter. These functions will need to loop internally in order to ensure that the right number of symbols is added:

```
full_line := n => l := ""; while n > 0: { n := n - 1; l := l + asterisk() }; return l + new_line()
hollow_line := n => l := ""; while n > 2: { n := n - 1; l := l + space() }; return asterisk() + l + asterisk() + new_line()
```

Armed with the certainty that each of these functions works in full isolation from the rest (we can test them on their own as needed in case we had any doubts!), we can compose them together to build the final figure. Again, this is quite simply a composition of drawing the individual lines:

```
hollow_square := n => l := full_line(n); while n > 2: { n := n - 1; l := l + hollow_line(n) }; return l + full_line(n)
```

[[STATE TRACE OF PROGRAM]]
    
We could also use some of the functions we have defined so far, for example `full_line`, to implement a (reversed) triangle. This requires simply the right sequence of invocations, and not much more:

```
triangle := n => l := ""; while n > 0: { n := n - 1; l := l + full_line(n) }; return l
```

[[STATE TRACE OF PROGRAM]]

## Functions in Python
We can translate the constructs we have seen so far into Python. Unfortunately Python offers a slightly more complex view of functions than the one we presented so far. Python allows us to define lambda expressions and assign them to variables, but unfortunately with an arbitrarily very restrictive definition of lambda expressions due to some peculiar philosophical motivations.

Moreover, in order to define functions without the arbitrary restrictions, then we need to assign them to a constant (a variable that can only be assigned once) by using a special syntax (which should not be needed were the restrictions on lambda expressions lifted) based on the keyword `def`. Fortunately, the `return` keyword works as expected.

For example, we could define a function to draw a line of asterisks as:

In [2]:
def draw_line(n):
    l = ""
    while n > 0:
        l = l + "*"
        n = n - 1
    return l
print(draw_line(5))

*****


Notice that in Python the definition and the naming of a function happen together at the same time, forcing us to treat a function and its name as a single unit of code, which should not really be the case.

Lambda expressions can also be defined in Python, but they will only contain an expression as their body which is then automatically returned right away, since a lambda in Python cannot contain multiple statements.

An example of such a lambda expression would be:

In [3]:
f = lambda x: x + 1
print(f(10))

11


Notice that the lambda expression above is, effectively, just a shorthand for

```
def f(x):
  return x + 1
```

## Functions inside expressions
As a closing note, let us now consider in more depth the relationship between function calls and expressions. When we first introduced expressions, we defined them in such a way that evaluation of an expression could not change the state. We now want to extend our expressions semantics so that we can also write an expression such as `f(x) + g(y)`, where `f` and `g` are functions. Unfortunately, evaluating a function might change the state. This means that our old way of tackling expressions, that is allowing them to only read, but not write to the state, will not work.

We must therefore reformulate the expression evaluation rules so that they can change the state. For this reason, all `eval_expr` rules that we defined so far will now be turned into `eval`, so that they may return a changed state instead of only the new expression for further computation.

As an example, consider the rules for evaluating a sum. These rules were once based on $\text{eval_expr}$, and read the state without changing it (by returning a new state):

- $\text{eval_expr(<E1 + E2>, S)} \rightarrow \text{<E1' + E2>}$ where $\text{eval_expr(<E1>, S)} \rightarrow \text{<E1'>}$
- $\text{eval_expr(<C + E>, S)} \rightarrow \text{<C + E'>}$ where $\text{eval_expr(<E>, S)} \rightarrow \text{<E'>}$ and $\text{C}$ is a (constant) value
- $\text{eval_expr(<C1 + C2>, S)} \rightarrow \text{<C>}$ where $\text{C1 + C2} \rightarrow \text{C}$ and $\text{C1, C2}$ are (constant) values

Now, these expressions rules will simply become part of the $\text{eval}$ rules, so that the evaluation of partial expressions may change the state:

- $\text{eval(<E1 + E2>, S)} \rightarrow \text{<E1' + E2>, S'}$ where $\text{eval(<E1>, S)} \rightarrow \text{<E1'>, S'}$
- $\text{eval(<C + E>, S)} \rightarrow \text{<C + E'>, S'}$ where $\text{eval(<E>, S)} \rightarrow \text{<E'>, S'}$ and $\text{C}$ is a (constant) value
- $\text{eval(<C1 + C2>, S)} \rightarrow \text{<C>, S}$ where $\text{C1 + C2} \rightarrow \text{C}$ and $\text{C1, C2}$ are (constant) values

This way, everything is uniformed to $\text{eval}$, and can change the state. This is useful for contexts such as function invocations inside expressions, or function invocations inside function invocations (`f(g(x))`).